# Subsections Issue

## Setup

### Imports

In [1]:
from typing import List
from pathlib import Path
from pprint import pprint
import re
import sys

import pandas as pd
import xlwings as xw

from buffered_iterator import BufferedIterator
import text_reader as tp
from sections import Rule, RuleSet, SectionBreak, ProcessingMethods, Section

### Logging

In [2]:
import logging
logging.basicConfig(format='%(name)-20s - %(levelname)s: %(message)s')
#logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('Text Processing')
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)

### Initialize 2-line Section Tests

#### 2-line Section Source

In [3]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'StartSection Name: B',
    'EndSection Name: B',
    'More text to be ignored',
    ]

## Iterator Options

### Two line section *StartSection*
- Start *Before* `StartSection`
- End *After* `EndSection`

In [4]:
two_line_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After')
    )

#### Using a regular iterator

In [5]:
test_iter = iter(GENERIC_TEST_TEXT)
pprint(two_line_section.read(test_iter))

pprint(two_line_section.read(test_iter))

['StartSection Name: A', 'EndSection Name: A']
[]


![Good](../examples/Valid.png) A regular iterator is missing the buffer, so *Step Back* calls can loose future items.
- The second call to `pprint(two_line_section.read(test_iter))` returns an empty list because the second *StartSection* line is lost.
  
|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`[]`|`[]`|

#### Using a *BufferedIterator* source.

In [6]:
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(two_line_section.read(test_iter))
pprint(two_line_section.read(test_iter))
pprint(two_line_section.read(test_iter))

['StartSection Name: A', 'EndSection Name: A']
['StartSection Name: B', 'EndSection Name: B']
[]


![Good](../examples/Valid.png) Supplying a *BufferedIterator* preserves the *Step Back* item.
- The Third call to `pprint(two_line_section.read(test_iter))` returns an empty list because a third *StartSection* is not found.

|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|
|`[]`|`[]`|

# Simple sections experimenting with start and end settings

#### 2-line Section Source

In [7]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'StartSection Name: B',
    'EndSection Name: B',
    'More text to be ignored',
    ]

## `end_on_first_item` Tests

The `end_on_first_item` parameter in a section definition determines whether the 
`end_section` break tests are applied to the first line in a section.
`end_on_first_item=True` does not *force* the section to complete after a single 
line.  It only makes it *possible* to stop after the first line.

The value of this parameter is that repeat sections do not require distinct
starting and ending sentinels. With `end_on_first_item=False` (the default) the 
same SectionBreak can be applied to both `start_section` and `end_section`.  
In this case, the second occurrence of the section begins immediately at the 
end of the previous section.

In [8]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',
    'EndSection B',
    'More text to be ignored',
    ]

#### Setting `end_on_first_item=False` (the default)
- Using identical `start_section` and `end_section`:
    > `start_section=SectionBreak('StartSection', break_offset='Before')`
    > `end_section=SectionBreak('StartSection', break_offset='Before')`

- Do not test first line of section (the default).
    > `end_on_first_item=False` 

In [9]:
start_sub_section = Section(
    section_name='StartSubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    end_on_first_item=False
    )

pprint(start_sub_section.read(GENERIC_TEST_TEXT))

['StartSection A', 'EndSection A']


![Good](../examples/Valid.png) The first section is returned as a list.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          ['StartSection A', 'EndSection A']
        </code></td>
        <td><code>
          ['StartSection A', 'EndSection A']
        </code></td></tr>
</table>

#### Defining a top section with a repeating subsection
- Using identical `start_section` and `end_section`:
    > `start_section=SectionBreak('StartSection', break_offset='Before')`
    > `end_section=SectionBreak('StartSection', break_offset='Before')`

- Do not test first line of section (the default).
    > `end_on_first_item=False` 

In [10]:
repeating_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('More text to be ignored', break_offset='Before'),
    subsections=start_sub_section
    )
pprint(repeating_section.read(GENERIC_TEST_TEXT))

[['StartSection A', 'EndSection A'], ['StartSection B', 'EndSection B']]


![Good](../examples/Valid.png) Both subsections are returned as a list of lists.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
          ['StartSection A', 'EndSection A'],<br>
           ['StartSection B', 'EndSection B']<br>
          ]
        </code></td>
        <td><code>
          [<br>
          ['StartSection A', 'EndSection A'],<br>
           ['StartSection B', 'EndSection B']<br>
          ]
        </code></td></tr>
</table>

- Using the same section definition, except allow testing of the first line of 
  the section.
    > `end_on_first_item=True` 

In [11]:
start_sub_section = Section(
    section_name='StartSubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    end_on_first_item=True
    )

pprint(start_sub_section.read(GENERIC_TEST_TEXT))

[]


![Good](../examples/Valid.png) The ending SectionBreak triggers on the same 
item that triggers the start of the section.  This will always result in an 
empty section.

|Expected|Actual|
|-|-|
|[]|[]|

#### Single Line Section.
- Using the same sentinel *('EndSection')*, but different `break_offset`.
    > `start_section=SectionBreak('EndSection', break_offset='Before')`
    > `end_section=SectionBreak('EndSection', break_offset='After')`

- Allow testing of the first line of section.
    > `end_on_first_item=True` 

In [12]:
end_sub_section = Section(
    section_name='EndSubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    end_on_first_item=True
    )

pprint(end_sub_section.read(GENERIC_TEST_TEXT))

['EndSection A']


![Good](../examples/Valid.png) Single line section.
- Starts *Before* **EndSection** 
- Ends *After* **EndSection** (the same line)

|Expected|Actual|
|-|-|
|['EndSection A']|['EndSection A']|

#### Same section definition as above except with `end_on_first_item=False` 

In [13]:
end_sub_section = Section(
    section_name='EndSubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    end_on_first_item=False
    )

pprint(end_sub_section.read(GENERIC_TEST_TEXT))

['EndSection A', 'StartSection B', 'EndSection B']


![Good](../examples/Valid.png) Section continues until **After** next 
*EndSection* is found

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          ['EndSection Name: A',<br> 
          'StartSection Name: B',<br>
          'EndSection Name: B']
        </code></td>
        <td><code>
          ['EndSection Name: A',<br>
          'StartSection Name: B',<br>
          'EndSection Name: B']
        </code></td></tr>
</table>

## Multiple Single line sections
### Single line section *StartSection*
> - Start *Before* `StartSection`
> - End *Before* `EndSection`

In [14]:
sub_section1 = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='Before')
    )

test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section1.read(test_iter))
pprint(sub_section1.read(test_iter))
pprint(sub_section1.read(test_iter))

['StartSection A']
['StartSection B']
[]


![Good](../examples/Valid.png) Single line `StartSections` are returned
|Expected|Actual|
|-|-|
|`['StartSection Name: A']`|`['StartSection Name: A']`|
|`['StartSection Name: B']`|`['StartSection Name: B']`|
|`[]`|`[]`|

### Defining a section that calls `sub_section1`

In [15]:
full_section = Section(
    section_name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[sub_section1]
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection A'], ['StartSection B']]


![Good](../examples/Valid.png) Both *StartSection* lines are found.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
        [<br>
          ['StartSection Name: A'],<br>
          ['StartSection Name: B'],<br>
        ]</code></td>
        <td><code>
        [<br>
          ['StartSection Name: A'],<br>
          ['StartSection Name: B'],<br>
        ]</code></td></tr>
</table>

### Multiple Attempts to define Single line *EndSection* 
#### Attempt #1. Copy *StartSection* design
> - Start *Before* `EndSection`
> - End *Before* `StartSection`


In [16]:
sub_section2a = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2a.read(test_iter))
pprint(sub_section2a.read(test_iter))
pprint(sub_section2a.read(test_iter))

['EndSection A']
['EndSection B', 'More text to be ignored']
[]


![Good](../examples/Valid.png) The result is expected, but not desired.
- This doesn't work well because there isn't another *StartSection* after the last it *EndSection*.
- As a result, it includes the unwanted text after the last *EndSection*.
 
|Expected|Actual|
|-|-|
|`['EndSection Name: A']`|`['EndSection Name: A']`|
|`['EndSection Name: B', 'More text to be ignored']`|`['EndSection Name: B', 'More text to be ignored']`|
|`[]`|`[]`|

#### Attempt #2
> - Start *Before* `EndSection`
> - End *After* `EndSection`

- This doesn't work because The first line is not tested so it only breaks after the second occurrence of *EndSection*.

In [17]:
sub_section2b = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2b.read(test_iter))
pprint(sub_section2b.read(test_iter))
pprint(sub_section2b.read(test_iter))

['EndSection A', 'StartSection B', 'EndSection B']
[]
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B']`|`['EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B']`|
|`[]`|`[]`|
|`[]`|`[]`|


#### Attempt #3
> - Start *Before* `EndSection`
> - End *After* `EndSection`
> - Enable testing of first item

- This should work because the first line should be tested and trigger a break.

In [18]:
sub_section2c = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2c.read(test_iter))
pprint(sub_section2c.read(test_iter))
pprint(sub_section2c.read(test_iter))

['EndSection A']
['EndSection B']
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['EndSection Name: A']`|`['EndSection Name: A']`|
|`['EndSection Name: B']`|`['EndSection Name: B']`|
|`[]`|`[]`|


#### Attempt #3.5
> - Start *Before* `EndSection`
> - End *Before* `EndSection`
> - Enable testing of first item

- This should __<u>not</u>__ work because the first line should be tested and 
  trigger a break before returning anything.

In [19]:
sub_section2cc = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='Before'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2cc.read(test_iter))
pprint(sub_section2cc.read(test_iter))
pprint(sub_section2cc.read(test_iter))

[]
[]
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`[]`|`[]`|
|`[]`|`[]`|
|`[]`|`[]`|


#### Attempt #4
> - Start *Before* `EndSection`
> - End is `True` (Always Break)
> - Enable testing of first item

- This should work because the first line should be tested and always trigger a break.

In [20]:
sub_section2d = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='After'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2d.read(test_iter))
pprint(sub_section2d.read(test_iter))
pprint(sub_section2d.read(test_iter))

['EndSection A']
['EndSection B']
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['StartSection Name: A']`|`['EndSection Name: A']`|
|`['StartSection Name: B']`|`['EndSection Name: B']`|
|`[]`|`[]`|

#### Attempt #5
> - Start *Before* `EndSection`
> - End is `True` (Always Break)
> - Don't enable testing of first item

- This should <u>not</u> work because the break should trigger on the second line.

In [21]:
sub_section2e = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='After')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2e.read(test_iter))
pprint(sub_section2e.read(test_iter))
pprint(sub_section2e.read(test_iter))

['EndSection A', 'StartSection B']
['EndSection B', 'More text to be ignored']
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['EndSection Name: A', 'StartSection Name: B']`|`['EndSection Name: A', 'StartSection Name: B']`|
|`['EndSection Name: B', 'More text to be ignored']`|`['EndSection Name: B', 'More text to be ignored']`|
|`[]`|`[]`|

**`end_section=SectionBreak(True)` defaults to `break_offset='Before'`**

#### Attempt #6
> - Start *Before* `EndSection`
> - End *After* ___`True`___ (Always Break)
> - Don't enable testing of first item

- This should <u>not</u> work because the break should trigger *After* the second line.

In [22]:
sub_section2f = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='After')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2f.read(test_iter))
pprint(sub_section2f.read(test_iter))
pprint(sub_section2f.read(test_iter))

['EndSection A', 'StartSection B']
['EndSection B', 'More text to be ignored']
[]


![Good](../examples/Valid.png)
|Expected|Actual|
|-|-|
|`['EndSection Name: A', 'StartSection Name: B']`|`['EndSection Name: A', 'StartSection Name: B']`|
|`['EndSection Name: B', 'More text to be ignored']`|`['EndSection Name: B', 'More text to be ignored']`|
|`[]`|`[]`|

#### Attempt #7
> - Start *Before* `EndSection`
> - Enable testing of first item
> - No End setting

- This should start with the first *EndSection* line and not stop until the end of the iterator.

In [23]:
sub_section2g = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2g.read(test_iter))
pprint(sub_section2g.read(test_iter))
pprint(sub_section2g.read(test_iter))

['EndSection A', 'StartSection B', 'EndSection B', 'More text to be ignored']
[]
[]


![Good](../examples/Valid.png)
|Expected|Actual|
|-|-|
|`['EndSection Name: A', ['StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']`|`['EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']`|
|`[]`|`[]`|
|`[]`|`[]`|


#### Single line section *EndSection* __Final Design__.
> - Start *Before* `EndSection`
> - End *Before* ___`True`___ (Always Break)
> - Don't enable testing of first item

- This should work because the break should trigger *Before* the second line.

In [24]:
sub_section2 = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2.read(test_iter))
pprint(sub_section2.read(test_iter))
pprint(sub_section2.read(test_iter))

['EndSection A']
['EndSection B']
[]


![Good](../examples/Valid.png) End breaks *Before* second line. 

|Expected|Actual|
|-|-|
|`['EndSection Name: A']`|`['EndSection Name: A']`|
|`['EndSection Name: B']`|`['EndSection Name: B']`|
|`[]`|`[]`|

### Combined Start and End subsections Single line section 
***sub_section1***
> - Start *Before* `StartSection`
> - End *Before* `EndSection`

***sub_section2*** 
> - Start *Before* `EndSection`
> - End *Before* ___`True`___ (Always Break)
> - Don't enable testing of first item

`subsections=[sub_section1, sub_section2]`

In [25]:

full_section = Section(
    section_name='Full',
    subsections=[sub_section1, sub_section2]
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[[['StartSection A'], ['EndSection A']], [['StartSection B'], ['EndSection B']]]


![Good](../examples/Valid.png) Tree list levels with single item in deepest level andm two items in each of the other two levels.

|Expected|Actual|
|-|-|
|`[['StartSection Name: A'], ['EndSection Name: A']]`|`[['StartSection Name: A'], ['EndSection Name: A']]`|
|`[['StartSection Name: B'], ['EndSection Name: B']]`|`[['StartSection Name: B'], ['EndSection Name: B']]`|

## `keep_partial` Tests
The `keep_partial` parameter in a section definition only applies when a list of subsections is given.  For all other cases the `keep_partial` parameter is ignored.
With a parent section definition that contains a list of subsections, a situation can occur where the parent section or the source ends before all of the subsections have been completed. 
- When `keep_partial=True` is set the final subsection list returned will be missing items for the subsections that did nopt get read.

- When `keep_partial=False` (the default) is set the incomplete final subsection list is not returned.

### Defining a Top Section that calls two single line subsections.

#### Two Subsection definitions 

In [26]:
start_sub_section = Section(
    section_name='StartSubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

end_sub_section = Section(
    section_name='EndSubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

#### Defining ***top_section*** 
- Contains an ending break:
    > `end_section=SectionBreak('ignored', break_offset='Before')`.

In [27]:
top_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, end_sub_section]
    )
pprint(top_section.read(GENERIC_TEST_TEXT))

[[['StartSection A'], ['EndSection A']], [['StartSection B'], ['EndSection B']]]


![Good](../examples/Valid.png) The Subsections are returned as lists of lists.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [['StartSection Name: A'],<br>
             ['EndSection Name: A']],<br>
             [['StartSection Name: B'],<br>
              ['EndSection Name: B']]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [['StartSection Name: A'],<br>
             ['EndSection Name: A']],<br>
             [['StartSection Name: B'],<br>
              ['EndSection Name: B']]<br>
          ]
        </code></td></tr>
</table>

#### Including unwanted text in between the start and end of subsection C

In [28]:
GENERIC_TEST_TEXT1 = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',
    'EndSection B',
    'StartSection C',
    'More text to be ignored',   # 'ignored' triggers end of top section
    'EndSection C',
    'Even more text to be ignored', 
    ]

pprint(top_section.read(GENERIC_TEST_TEXT1))

[[['StartSection A'], ['EndSection A']], [['StartSection B'], ['EndSection B']]]


![Good](../examples/Valid.png) Subsections A and B are returned Subsections C is dropped.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
             [['StartSection B'], ['EndSection B']]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], ['EndSection B']]<br>
          ]
        </code></td></tr>
</table>

#### Setting `keep_partial=False`
- This should be the default

In [29]:
top_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, end_sub_section],
    keep_partial=False
    )

pprint(top_section.read(GENERIC_TEST_TEXT1))

[[['StartSection A'], ['EndSection A']], [['StartSection B'], ['EndSection B']]]


![Good](../examples/Valid.png) The same result as not including `keep_partial=False`
- Subsections A and B are returned. Subsections C is dropped.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], ['EndSection B']]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], ['EndSection B']]<br>
          ]
        </code></td></tr>
</table>

#### Setting `keep_partial=True`

In [30]:
top_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, end_sub_section],
    keep_partial=True
    )

pprint(top_section.read(GENERIC_TEST_TEXT1))

[[['StartSection A'], ['EndSection A']],
 [['StartSection B'], ['EndSection B']],
 [['StartSection C'], []]]


![Bad](../examples/error.png) Expecting `['StartSection C']`, 
but not the following empty list.
- That may not be a bad thing.
- Just need to understand why it happens

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], ['EndSection B']]<br>
            [['StartSection C']]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], ['EndSection B']]<br>
            [['StartSection C'], []]<br>
          ]
        </code></td></tr>
</table>

##### Using same `keep_partial=True` setting with original source.

In [31]:
pprint(top_section.read(GENERIC_TEST_TEXT))

[[['StartSection A'], ['EndSection A']], [['StartSection B'], ['EndSection B']]]


![Good](../examples/Valid.png) `keep_partial=True` has no effect because both 
subsections A and B are completed. 

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [['StartSection Name: A'], ['EndSection Name: A']],<br>
            [['StartSection Name: B'], ['EndSection Name: B']]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [['StartSection Name: A'], ['EndSection Name: A']],<br>
            [['StartSection Name: B'], ['EndSection Name: B']]<br>
          ]
        </code></td></tr>
</table>

##### Using same `keep_partial=True` with only `end_sub_section`.

In [32]:
top_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[end_sub_section],
    keep_partial=True
    )

pprint(top_section.read(GENERIC_TEST_TEXT1))

[['EndSection A'], ['EndSection B']]


![Good](../examples/Valid.png) `keep_partial=True` has no effect because single 
subsections are processed differently. 
- Results in one less level of lists.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            ['EndSection Name: A'],<br>
            ['EndSection Name: B']<br>
          ]
        </code></td>
        <td><code>
          [<br>
            ['EndSection Name: A'],<br>
            ['EndSection Name: B']<br>
          ]
        </code></td></tr>
</table>

#### Dropping the corresponding *EndSection* for a *StartSection*.

In [33]:
GENERIC_TEST_TEXT2 = [
    'Text to be ignored',
    'StartSection A',
    'EndSection A',
    'StartSection B',  # Missing 'EndSection B',
    
    'StartSection C',
    'More text to be ignored',   # 'ignored' triggers end of top section
    'EndSection C',
    'Even more text to be ignored', 
    ]

In [34]:
top_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, end_sub_section],
    keep_partial=True
    )

pprint(top_section.read(GENERIC_TEST_TEXT2))

[[['StartSection A'], ['EndSection A']], [['StartSection B'], []]]


![Bad](../examples/error.png) Expecting `['StartSection B']`, 
but not the following empty list.
- 3rd section group never starts because **ignored** *Top Section* break line 
  occurs before next **EndSection**, so 2nd never finishes.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B']]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], []]<br>
          ]
        </code></td></tr>
</table>

In [35]:
top_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, end_sub_section],
    keep_partial=False
    )

pprint(top_section.read(GENERIC_TEST_TEXT2))

[[['StartSection A'], ['EndSection A']], [['StartSection B'], []]]


![Bad](../examples/error.png) Not expecting partial `['StartSection B']`.
- 2nd section group never finishes because **ignored** *Top Section* break 
  line occurs.  Partial subsection should have been dropped.

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [['StartSection A'], ['EndSection A']],<br>
            [['StartSection B'], []]<br>
          ]
        </code></td></tr>
</table>

### Including unwanted text in between subsections

In [36]:
GENERIC_TEST_TEXT1 = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'StartSection Name: B',
    'More text to be ignored', 
    'EndSection Name: B',
    'Even more text to be ignored', 
    ]

pprint(full_section.read(GENERIC_TEST_TEXT1))

[[['StartSection Name: A'], ['EndSection Name: A']],
 [['StartSection Name: B', 'More text to be ignored'], ['EndSection Name: B']]]


![Good](../examples/Valid.png) The *to be ignored* text between subsections is dropped.

|Expected|Actual|
|-|-|
|`[['StartSection Name: A'], ['EndSection Name: A']]`|`[['StartSection Name: A'], ['EndSection Name: A']]`|
|`[['StartSection Name: B'], ['EndSection Name: B']]`|`[['StartSection Name: B'], ['EndSection Name: B']]`|

## Hysteresis Bug Fixed  ![Good](../examples/Valid.png) 
When `full_section` contains 
`end_section=SectionBreak('ignored', break_offset='Before')`, repeated calls produce different effects.

#### Define `SubSection`: 
- Start *Before* `StartSection`
- End *After* `EndSection`

In [37]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before', name='SubSectionStart'),
    end_section=SectionBreak('EndSection', break_offset='After', name='SubSectionEnd')
    )

1. Clear Hysteresis by running `full_section` without setting `end_section`
2. Define `full_section` with 
`end_section=SectionBreak('ignored', break_offset='Before')`.

In [38]:
full_section = Section(
    section_name='Full',
    subsections=sub_section,
    keep_partial=False
    )
a = full_section.read(GENERIC_TEST_TEXT)
b = full_section.read(GENERIC_TEST_TEXT)

full_section = Section(
    section_name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=sub_section
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection A', 'EndSection A'], ['StartSection B', 'EndSection B']]


![Good](../examples/Valid.png) 
First time it runs as expected
- Both full subsections found. No empty lists afterwards.

|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|

In [39]:
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection A', 'EndSection A'], ['StartSection B', 'EndSection B']]


![Good](../examples/Valid.png)  Hysteresis Fixed
- Before bug fix, The second time, a line in the first subsection is missing.

|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|


## Check *Source* status
- Check ___Source___ status after section read.

In [40]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before', name='SubSectionStart'),
    end_section=SectionBreak('EndSection', break_offset='After', name='SubSectionEnd')
    )
full_section = Section(
    section_name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=sub_section
    )
a = full_section.read(GENERIC_TEST_TEXT)
print('previous_items')
pprint(list(full_section.source.previous_items))
print()
print('future_items')
pprint(list(full_section.source.future_items ))

previous_items
['StartSection A', 'EndSection A', 'StartSection B', 'EndSection B']

future_items
['More text to be ignored']


#### Expected source values
<table>
<thead>
  <tr>
    <th>Buffer</th>
    <th>Expected</th>
    <th>Actual</th>
    <th>Test</th>
  </tr>
</thead>
<tr><td><b>previous_items</b></td>
<td><code>['StartSection A', 'EndSection A',<br>
           'StartSection B', 'EndSection B']</code></td>
<td><code>['StartSection A', 'EndSection A',<br>
           'StartSection B', 'EndSection B']</code></td>
<td><img src="../examples/Valid.png" alt="Good"/></td></tr>
<tr><td><b>future_items</b></td>
<td><code>['More text to be ignored']</code></td>
<td><code>['More text to be ignored']</code></td>
<td><img src="../examples/Valid.png" alt="Good"/></td></tr>
</table>

## Check Context

***Context Standard Items***
<table>
<thead><th>Item Key</th><th>Value Description</th></thead>
<tr><td>Current Section</td>
<td>The assigned name of the section <i>self.section_name</i></td></tr>

<tr><td>Skipped Lines</td>
<td>A list of items in the source occurring before the first item of the section</td></tr>

<tr><td>Status</td>
<td>The current section scanning status one of:<ul>
    <li>Not Started
    <li>At section start
    <li>Scan In Progress
    <li>Break Triggered
    <li>Scan Complete
    <li>End of Source
    </ul></td></tr>
<tr><td>Break</td>
<td>The name assigned to the SectionBreak that most recently triggered. 
Before the section start this is not defined. 
While the section scan is in progress, this is the name of the StartSection. 
After the section ends, this is the name of the the EndSection.</td></tr>
<tr><td>
Event</td>
<td>The result from the SectionBreak that most recently triggered; the 
<i>TriggerEvent.test_value</i>.  
For more information see the TriggerEvent documentation.</td></tr>
</table>

- Compare `full_section` and `sub_section` ___Context___ after reading.


In [41]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'Text between sections',
    'StartSection Name: B',
    'EndSection Name: B',
    'More text to be ignored',
    ]

In [42]:
sub_section = Section(
    section_name='Sub-Section',
    start_section=SectionBreak('StartSection', break_offset='Before',
                               name='SubSectionStart'),
    end_section=SectionBreak('EndSection', break_offset='After', 
                             name='SubSectionEnd')
    )
full_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before', 
                             name='End Section'),
    subsections=sub_section
    )
a = full_section.read(GENERIC_TEST_TEXT, context={'Dummy': 'Blank1'})
print()
print('full_section context')
pprint(full_section.context)
pprint(full_section.scan_status)
print()
print('sub_section context')
pprint(sub_section.context)
pprint(sub_section.scan_status)


full_section context
{'Break': 'End Section',
 'Current Section': 'Top Section',
 'Dummy': 'Blank1',
 'Event': 'ignored',
 'Skipped Lines': [],
 'Status': 'Break Triggered'}
'Break Triggered'

sub_section context
{'Break': 'SubSectionStart',
 'Current Section': 'Sub-Section',
 'Dummy': 'Blank1',
 'Event': 'StartSection',
 'Skipped Lines': ['Text between sections'],
 'Status': 'End of Source'}
'End of Source'


**Context Tests**
<style type="text/css">
.tg th{font-weight:bold;text-align:center;vertical-align:middle}
.tg td{font-weight:normal;text-align:left;vertical-align:top}
.rh td{font-weight:bold;text-align:center;vertical-align:middle}
</style>

<table class="tg">
    <thead>
        <tr>
            <th rowspan="2" text-align="center">Key</th>
            <th colspan="3">Top Section</th>
            <th colspan="3">Sub-Section</th>
        </tr><tr>
            <th>Expected Value</th><th>Actual Value</th><th>Test</th>
            <th>Expected Value</th><th>Actual Value</th><th>Test</th>
        </tr>
    </thead>
        <tr><td class="rh">Current Section</td>
            <td>'Top Section'</td>
            <td>'Top Section'</td>
            <td><img src="../examples/Valid.png" alt="Good"/></td>
            <td>'Sub-Section'</td>
            <td>'Sub-Section'</td>
            <td><img src="../examples/Valid.png" alt="Good"/></td>
        </tr>
    <tr><td class="rh">Dummy</td>
        <td>'Blank1'</td>  
        <td>'Blank1'</td>
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>'Blank1'</td>  
        <td>'Blank1'</td>
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
    <tr><td class="rh">Skipped Lines</td>  
        <td>[]</td>
        <td>[]</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>['Text between sections']</td>
        <td>['Text between sections']</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
    <tr><td class="rh">Status</td>  
        <td>'Break Triggered'</td>
        <td>'Break Triggered'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>'End of Source'</td>
        <td>'End of Source'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
    <tr><td class="rh">Break</td>  
        <td>'End Section'</td>
        <td>'End Section'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>'SubSectionStart'</td>
        <td>'SubSectionStart'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
    <tr><td class="rh">Event</td>  
        <td>'ignored'</td>
        <td>'ignored'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>'StartSection'</td>
        <td>'StartSection'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
</table>

- ***Current Section*** is the name of the section given at the section definition.
    - For _full_section_,  `name='End Section'`
    - For _sub_section_,   `name='Sub-Section'`
- ***Dummy*** is a context element supplied at the time `full_section.read()` was 
  called. Since it is not one of the standard context items it is just along for 
  the ride unless a user defined function explicitly modifies it. 
    - The initially supplied value of ***Dummy*** = _'Blank1'_ is the same for 
      both _full_section_ and _sub_section_.
- ***Skipped Lines*** is the list of unused lines encountered before the beginning 
  of the section.
    - For _full_section_, start_section is not given so ***Skipped Lines*** is 
      an empty list.
    - For _sub_section_, ***Skipped Lines*** is the lines skipped before the 
      most recent sub-section: ['Text between sections']
- ***Status*** is the section state. 
    - _full_section_ has ended because the `end_section` break has triggered, so
      ***Status*** = 'Break Triggered'
    - _sub_section_ receives it's source from _full_section_, but 
      _full_section_ has ended.  _sub_section_ interprets this as and end to 
      it's source, so ***Status*** = 'End of Source'
- ***Break*** is the name of the most recent break triggered while the section is 
   active.
    - The `end_section` break of _full_section_ has triggered.  The name of 
      this SectionBreak was given as 'End Section', so 
      ***Break*** = 'End Section'.
    - The `end_section` break of _sub_section_ never triggered because it's 
      source ended first. The most recent section break for _sub_section_ was 
      the `start_section`, so ***Break*** = 'SubSectionStart'.
- ***Event*** is the Trigger specific value resulting from the last break.  For 
  string based breaks, ***Event*** is the string used to test for the break.
    - The `end_section` break of _full_section_ was triggered by the string 
      'ignored', so ***Event*** = 'ignored'.
    - The `start_section` break of _sub_section_ was triggered by the string 
      'StartSection', so ***Event*** = 'StartSection'.

**********

# DONE TO HERE

**********

### Initial Section and Sub-Section Definitions

- Only definition line is:<br>
`subsections=[sub_section]`

In [50]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['Text to be ignored',
  'StartSection Name: A',
  'EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) Results in all lines in one sub-list

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[['Text to be ignored',<br>
  'StartSection Name: A',
  'EndSection Name: A',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]
  </code></td><td><code>
  [['Text to be ignored',<br>
  'StartSection Name: A',
  'EndSection Name: A',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]
  </code></td></tr>
</table>



**********

## Section Break options

### Add start to Section Definition

- Section start **Before** *StartSection*

`start_section=SectionBreak('StartSection', break_offset='Before'),`

In [51]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A',
  'EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) Includes all lines after first *StartSection* in single sub-list.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[['StartSection Name: A',
  'EndSection Name: A',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]
  </code></td><td><code>
  [['StartSection Name: A',
  'EndSection Name: A',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]
  </code></td></tr>
</table>



### Add end to Section Definition

- Section start **Before** *StartSection*
- Section end **After** *EndSection*

```python
full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [52]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A']]


- ![Good](../examples/Valid.png) Includes both lines of first section in single sub-list.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[['StartSection Name: A',
  'EndSection Name: A']]
  </code></td><td><code>
  [['StartSection Name: A',
  'EndSection Name: A']]
  </code></td></tr>
</table>



### Define Muti-Section to Read Both Sections

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- Simple subsection
- Multi Section defines Full Section as Sub Section with no start or end (All lines)

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

multi_section = Section(
    section_name='Multi',
    subsections=[full_section] 
    )
```

In [53]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

multi_section = Section(
    section_name='Multi',
    subsections=[full_section] 
    )

pprint(multi_section.read(GENERIC_TEST_TEXT))

[[['StartSection Name: A', 'EndSection Name: A']],
 [['StartSection Name: B', 'EndSection Name: B']]]


- ![Good](../examples/Valid.png)Includes both lines of each section in its own sub-list.
- ??? Is this expected???  ![Bad](../examples/error.png)

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>
[[['StartSection Name: A', 'EndSection Name: A']],
 [['StartSection Name: B', 'EndSection Name: B']]]
</code></td></tr>
</table>

### Set Same Start and End Breaks for Section

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **After** *StartSection*
- Simple subsection
- Multi Section defines Full Section as Sub Section with no start or end (All lines)

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    subsections=[sub_section] 
    )

multi_section = Section(
    section_name='Multi',
    subsections=[full_section] 
    )
```

In [54]:
full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before')
    )

multi_section = Section(section_name='Multi',
    subsections=[full_section] 
    )

pprint(multi_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A', 'Text between sections'],
 ['StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']]


- ![Good](../examples/Valid.png) Without SubSection, it works as expected.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>
[['StartSection Name: A', 'EndSection Name: A'],<br>
 ['StartSection Name: B', 'EndSection Name: B',<br>
 'More text to be ignored']]
  </code></td><td><code>
[['StartSection Name: A', 'EndSection Name: A'],<br>
 ['StartSection Name: B', 'EndSection Name: B',<br>
 'More text to be ignored']]
  </code></td></tr>
</table>

In [55]:
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before')
    )

multi_section = Section(
    section_name='Multi',
    subsections=[full_section] 
    )

pprint(multi_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A', 'Text between sections'],
 ['StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']]


In [76]:
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    subsections=[sub_section] 
    )

source = iter(GENERIC_TEST_TEXT)
pprint(full_section.read(source))

[['StartSection Name: A', 'EndSection Name: A', 'Text between sections']]


In [58]:
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    subsections=[sub_section] 
    )

source = iter(GENERIC_TEST_TEXT)
pprint(full_section.read(source, start_search=False))
pprint(full_section.read(source, start_search=False))
pprint(full_section.read(source, start_search=False))

[['Text to be ignored']]
[['EndSection Name: A', 'Text between sections']]
[['EndSection Name: B', 'More text to be ignored']]


In [59]:
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    subsections=[sub_section] 
    )

source = iter(GENERIC_TEST_TEXT)
pprint(full_section.read(source, start_search=False))
pprint(full_section.read(source))
pprint(full_section.read(source, start_search=False))

[['Text to be ignored']]
[['StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']]
[]


In [60]:
sub_section = Section(section_name='SubSection',end_on_first_item=True)

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    subsections=[sub_section],end_on_first_item=True
    )

source = iter(GENERIC_TEST_TEXT)
pprint(full_section.read(source, start_search=False))
pprint(full_section.read(source))
pprint(full_section.read(source, start_search=False))

[['Text to be ignored']]
[]
[['EndSection Name: B', 'More text to be ignored']]


In [61]:
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    subsections=[sub_section] 
    )

multi_section = Section(
    section_name='Multi',
    subsections=[full_section] 
    )

pprint(multi_section.read(GENERIC_TEST_TEXT))

[[['StartSection Name: A', 'EndSection Name: A', 'Text between sections']],
 [['StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']]]


- ![Bad](../examples/error.png) Appears to work now

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code></code></td></tr>
</table>

**********

## SubSection Break Options

### Add *End on First* to Sub-Section

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection **End On First**

```python
sub_section = Section(section_name='SubSection',
    end_on_first_item=True,
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [62]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='After')
    end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in `[['StartSection Name: A', 'EndSection Name: A']]`.

<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>[['StartSection Name: A', 'EndSection Name: A']]</code></td>
<td><code>[['StartSection Name: A', 'EndSection Name: A']]</code></td></tr>
</table>

### Add *End* as **True** to Sub-Section

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection end is **True**  Should be the same as **End On First**

```python
sub_section = Section(section_name='SubSection',
    end_section=SectionBreak(True)
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [63]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='After')
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A']]


#### What should `end_section=SectionBreak(True)` result in?
- ![Good](../examples/Valid.png) Results in `['StartSection Name: A'], []`
- ??? What is expected??? ![Bad](../examples/error.png)
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>[['StartSection Name: A']]</code></td></tr>
</table>

### Add *Start* to Sub-Section

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **Before** *EndSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before')
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [64]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before')
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

[]


- ![Bad](../examples/error.png) Results in blank sub-list.
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>[[]]</code></td></tr>
</table>

### Change Start Sub-Section to **After**

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **Before** *EndSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before')
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [65]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='After')
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

[]


- ![Bad](../examples/error.png) Results in blank sub-list.
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>[[]]</code></td></tr>
</table>

### Add start to SubSection Definition
> `start_section=SectionBreak('EndSection', break_offset='Before')`

In [66]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Bad](../examples/Error.png) - ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>
[['EndSection Name: A',<br>
  'StartSection Name: B',
  'EndSection Name: B',<br>
  'More text to be ignored']]
  </code></td></tr>
</table>

### Add *Start After* to Section Definition
> `start_section=SectionBreak('StartSection', break_offset='After')`

In [67]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) Results in one line section
- ![Bad](../examples/Error.png) - ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>
[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]
  </code></td></tr>
</table>

### Add *Start After StartSection* to Section Definition and *Start Before End Section* to SubSection Definition
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before')`
    

In [68]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Bad](../examples/error.png) - ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>
[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]
  </code></td></tr>
</table>

### Add *Start After StartSection* to Section Definition and *Start Before End Section* to SubSection Definition and set *End On First Line* for SubSection
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_on_first_item=True,`
    

In [69]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Bad](../examples/Error.png) - ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>
[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]
  </code></td></tr>
</table>

### Add *Start After StartSection* to Section Definition, and for SubSection Definition, set *Start* to  *Before EndSection* and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [70]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A'], ['EndSection Name: B']]


- ![Good](../examples/Valid.png) Results in one line section
- ![Bad](../examples/Error.png) - ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>
[['EndSection Name: A'], ['EndSection Name: B'], []]
  </code></td></tr>
</table>

### Add *Start After StartSection* and *End Before EndSection* to Section Definition, and for SubSection Definition, set *Start* to  *Before EndSection* and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`
    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [71]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in one line section
- ![Bad](../examples/Error.png) - ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>
[['EndSection Name: A']]
  </code></td></tr>
</table>

### Add *Start __Before__ StartSection* and *End Before EndSection* to Section Definition, and for SubSection Definition, set *Start* to  *Before EndSection* and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`
    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [72]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[]


- ![Bad](../examples/Error.png) Results in empty list of lists
- ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>[[]]</code></td></tr>
</table>

### Add *Start __Before__ StartSection* and *End Before EndSection* to Section Definition, and for SubSection Definition, set *Start* to *Before EndSection*, *End* to *`True` (Always Break)* and *Keep Partial* to *`True`*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`<br>
> `end_section=SectionBreak(True),`<br>
> `keep_partial=True,`
    

In [73]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[]


- ![Bad](../examples/Error.png) Results in empty list of lists
- ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code></code></td>
<td><code>[[]]</code></td></tr>
</table>

### Add *Start __Before__ StartSection* and *End Before EndSection* to Section Definition, and don't set any SectionBreaks for SubSection Definition, 
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
    

In [74]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in Two line section

- ??? Is this incorrect or is this expected???
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>[['StartSection Name: A', 'EndSection Name: A']]</code></td>
<td><code>[['StartSection Name: A', 'EndSection Name: A']]</code></td></tr>
</table>

### Add *Start __After__ StartSection* and *End Before EndSection* to Section Definition, and don't set any SectionBreaks for SubSection Definition, 
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
    

In [75]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'Text between sections',
  'StartSection Name: B',
  'EndSection Name: B']]


- ![Good](../examples/Valid.png) Results in one line section
  
<table><thead><th>Expected</th><th>Actual</th></thead>
<tr><td><code>[['EndSection Name: A']]</code></td>
<td><code>[['EndSection Name: A']]</code></td></tr>
</table>

## Three line sections

In [ ]:
GENERIC_TEST_TEXT2 = [
    'Text to be ignored',
    'StartSection A',
    'MiddleSection A',
    'EndSection A',
    'StartSection B',
    'MiddleSection B',
    'EndSection B',
    'StartSection C',
    'MiddleSection C',
    'EndSection C',
    'Even more text to be ignored', 
    ]

#### Single line subsection definitions 

In [ ]:
start_sub_section = Section(
    section_name='StartSubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

middle_sub_section = Section(
    section_name='MiddleSubSection',
    start_section=SectionBreak('MiddleSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

end_sub_section = Section(
    section_name='EndSubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )

#### Defining ***three_part_section*** 
- Contains an ending break:
    > `end_section=SectionBreak('ignored', break_offset='Before')`.

- Contains 3 subsections:
    > `[start_sub_section, middle_sub_section, end_sub_section]`

In [ ]:
three_part_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, middle_sub_section, end_sub_section]
    )
pprint(three_part_section.read(GENERIC_TEST_TEXT2))

[[['StartSection A'], ['MiddleSection A'], ['EndSection A']],
 [['StartSection B'], ['MiddleSection B'], ['EndSection B']],
 [['StartSection C'], ['MiddleSection C'], ['EndSection C']]]


![Good](../examples/Valid.png) All 3 sections and subsections are completed. 

<table>
    <thead><th>Expected</th><th>Actual</th></thead>
    <tr>
        <td><code>
          [<br>
            [
              ['StartSection Name: A'],<br> 
              ['MiddleSection A'],<br> 
              ['EndSection Name: A']
            ],<br>
            [
              ['StartSection Name: B'],<br> 
              ['MiddleSection A'],<br> 
              ['EndSection Name: B']
            ]<br>
            [
              ['StartSection Name: C'],<br> 
              ['MiddleSection C'],<br> 
              ['EndSection Name: C']
            ]<br>
          ]
        </code></td>
        <td><code>
          [<br>
            [
              ['StartSection Name: A'],<br> 
              ['MiddleSection A'],<br> 
              ['EndSection Name: A']
            ],<br>
            [
              ['StartSection Name: B'],<br> 
              ['MiddleSection A'],<br> 
              ['EndSection Name: B']
            ]<br>
            [
              ['StartSection Name: C'],<br> 
              ['MiddleSection C'],<br> 
              ['EndSection Name: C']
            ]<br>
          ]
        </code></td></tr>
</table>

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection **End On First**

```python
sub_section = Section(section_name='SubSection',
    end_on_first_item=True,
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [ ]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='After')
    end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A']]


In [ ]:
top_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[start_sub_section, end_sub_section]
    )
pprint(top_section.read(GENERIC_TEST_TEXT))

[[['StartSection Name: A'], ['EndSection Name: A']],
 [['StartSection Name: B'], ['EndSection Name: B']]]


In [ ]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='After')
    )

In [ ]:
full_section = Section(
    section_name='Full',
    subsections=sub_section,
    keep_partial=True
    )